# Exercise 2

https://www.sciencedirect.com/science/article/abs/pii/S0377221705005643

In [1]:
import simpy as sp
import numpy as np
import pandas as pd
import seaborn as sns
import random

RANDOM_SEED = 42
n = 2 # number of servers in the simulation
T_ARRIVAL = 7 # arrival of a new customer
SIM_TIME = 10100 # Simulation length in minutes
rho = 0.95
mu = .75
SERVICETIME = 1/mu # average minutes to complete a task

wtime = 100
btime = 100
mtime = 400

# def monitor(data, resource):
#     """This is a monitoring callback"""
#     item = [
#         resource._env.now,
#         resource.count,
#         len(resource.queue)
#     ]
#     data.append(item)

class Server(object):
    
    def __init__(self, env, num_server, servicetime):
        
        self.env = env
        self.server = sp.Resource(env, num_server)
        self.servicetime = servicetime
        
    def service(self, customer):
        # need to add stochasticity to this
        
        yield self.env.timeout(randomSample(SERVICETIME))
        
def randomSample(rate):
    
    return np.random.exponential(rate)
        
def client(env, name, sv, data):
    
    arrival = env.now
    #print("%s arrives at %.2f." %(name, arrival))
    
    if env.now%500 > 100:
        data[0].append(len(sv.server.queue))
    
    with sv.server.request() as request:
        
        startwaiting = env.now
        yield request
        waitingtime = env.now-startwaiting
        
        service = env.now
#         print("%s is processed at %.2f." % (name, service))
        
        if env.now%500 > 100:
            data[0].append(len(sv.server.queue))
            data[1].append(waitingtime)
        
        yield env.process(sv.service(name))
        
#         print("%s is finished at %.2f." %(name, env.now))
        
        
def setup(env, num_servers, servicetime, t_arrival, data):
    
    server = Server(env, num_servers, servicetime)
    
#     # create 4 customers
#     for i in range(4*num_servers):
#         data[0] += 1
#         env.process(client(env, 'Client %d' % data[0], server, data))


    # make more customers while the simulation is running
    i = 0
    while True:
        i+=1
        yield env.timeout(randomSample(1/lambd))
        
        env.process(client(env, 'Client %d' % i, server, data))

        
def runSimulation(serverNumber, data):
    
    #random.seed(RANDOM_SEED)

    # create the enviroment and start the setup process

    env = sp.Environment()
    env.process(setup(env, serverNumber, SERVICETIME, T_ARRIVAL, data))

    env.run(SIM_TIME)

#     print("The total waiting time was %.2f" %(data[2]))
#     print("There were %i clients, %i of who(m?) had to wait." %(data[0],data[1]))
#     print("The average waiting time was %.2f" %(data[2]/data[0]))
    
def calcurmv(xj, sj, xc, j):
    xj1 = xj + ((xc-xj)/(j+1))
    sj1 = (((1-(1/j))*(sj**2)) + (j+1)*((xj1-xj)**2))**0.5
    return xj1, sj1

def checkstop(s,k):
    l = 1.0
    za = 2.975
    if (2*za*s/(k)**0.5)<l:
        return True
    else:
        return False

In [2]:
data = [[],[]]

waitingdata



l = 0.01
za = 2.975

n = 2
lambd = rho * mu * n

runSimulation(n, data)

quelen = np.array(data[0])
stime = np.array(data[1])

xqm = np.mean(quelen)
xsm = np.mean(stime)

sq = np.var(quelen)
ss = np.var(stime)

i = 1

while True:
    print("Running..", i, sq, ss)
    
    data = [[],[]]
    runSimulation(n, data)
    
    quelen = np.array(data[0])
    stime = np.array(data[1])
    xqt = np.mean(quelen)
    xst = np.mean(stime)
    
    xqm, sq = calcurmv(xqm, sq, xqt, i)
    xsm, ss = calcurmv(xsm, ss, xst, i)
    
    if i>=100:
        #print("Checkstop can activate")
        if i%50 == 0:
            checkSS = (2*za*sq/(i)**0.5)
            checkSQ = (2*za*ss/(i)**0.5)
            print(checkSS)
            print(checkSQ)
        if checkstop(sq, i) and checkstop(ss, i):
            print("We in this bithc")
            break
    i+=1
    

Running.. 1 346.99306213401326 163.13449268223397
Running.. 2 0.7924466521423432 0.6946710378046964
Running.. 3 7.620014964016345 5.326434721162229
Running.. 4 6.58118746911948 4.593721019761403
Running.. 5 5.700081110807632 3.980706564170045
Running.. 6 5.249125822935012 3.676096544340938
Running.. 7 5.394627663815777 3.7996234157040365
Running.. 8 5.708975324362311 3.998062576736635
Running.. 9 5.63892367140378 3.920889007045206
Running.. 10 5.468888205166758 3.7977713496689205
Running.. 11 5.401405586720564 3.7393426937466008
Running.. 12 5.264664039228343 3.6459123231955686
Running.. 13 5.060759554276151 3.506646466260301
Running.. 14 5.007123924611442 3.462508851624776
Running.. 15 5.194868768620707 3.582544460902914
Running.. 16 5.095894943308772 3.5020060296926028
Running.. 17 4.935962827823774 3.3921424580397472
Running.. 18 4.799067711623417 3.298909460892406
Running.. 19 5.032915477901042 3.4188252031111874
Running.. 20 4.90153089014671 3.3296224753731316
Running.. 21 5.07412

Running.. 164 5.554287746719501 3.790034188588313
Running.. 165 5.537721331426555 3.7787841560694666
Running.. 166 5.524173286842204 3.7700382918501565
Running.. 167 5.508254236135539 3.7592066265595436
Running.. 168 5.494313736557142 3.749398922736782
Running.. 169 5.483036590123473 3.7416358151095044
Running.. 170 5.493664557212266 3.7485649763023763
Running.. 171 5.482244684979813 3.7411905010723103
Running.. 172 5.495797395884425 3.7508721613911793
Running.. 173 5.494597876160229 3.750315073906818
Running.. 174 5.488288381232782 3.745683551295282
Running.. 175 5.480993134929212 3.7414424354554345
Running.. 176 5.4679185574081455 3.7322481934856993
Running.. 177 5.455350387817623 3.7238352569030075
Running.. 178 5.442871000272581 3.7151812838646503
Running.. 179 5.440907538050444 3.7138011769542363
Running.. 180 5.431523298159438 3.7073205284012816
Running.. 181 5.435023605271433 3.710171777681733
Running.. 182 5.420641531572807 3.700494739052775
Running.. 183 5.408908207706922 3.69

Running.. 324 5.441167208925418 3.7055541750954495
Running.. 325 5.519757589630007 3.762095041218698
Running.. 326 5.51577061932289 3.7590174739158257
Running.. 327 5.514108110145217 3.7578980867649747
Running.. 328 5.50703237228229 3.7528896307944355
Running.. 329 5.499091816740021 3.7474642629487924
Running.. 330 5.494533652671222 3.7441963387666517
Running.. 331 5.499604218438288 3.7468876686717825
Running.. 332 5.545324982205572 3.781802551496101
Running.. 333 5.540211318137082 3.7783306208746397
Running.. 334 5.532354698160106 3.7730305546768954
Running.. 335 5.541269002377128 3.779322495750449
Running.. 336 5.535876545832899 3.7754480578183425
Running.. 337 5.540301832791529 3.7784804908914738
Running.. 338 5.53239979019896 3.7730734186260078
Running.. 339 5.530275240643976 3.7716393702566746
Running.. 340 5.5373679656278325 3.77740015045699
Running.. 341 5.545679520332345 3.7832332417838708
Running.. 342 5.538550443348227 3.7782256231460494
Running.. 343 5.531197071372831 3.7733

KeyboardInterrupt: 

the link https://simpy.readthedocs.io/en/latest/examples/carwash.html

In [93]:
data = [0,0,0]

n = 1
lambd = rho * mu * n

runSimulation(n, data)


Server is up
Client 1 arrives at 0.00.
Client 2 arrives at 0.00.
Client 3 arrives at 0.00.
Client 4 arrives at 0.00.
Client 1 is processed at 0.00.
Client 1 is finished at 0.77.
Client 2 is processed at 0.77.
Client 5 arrives at 0.79.
Client 6 arrives at 0.83.
Client 2 is finished at 1.76.
Client 3 is processed at 1.76.
Client 7 arrives at 1.79.
Client 8 arrives at 2.40.
Client 9 arrives at 2.90.
Client 10 arrives at 3.45.
Client 11 arrives at 5.14.
Client 3 is finished at 5.20.
Client 4 is processed at 5.20.
Client 4 is finished at 5.75.
Client 5 is processed at 5.75.
Client 12 arrives at 6.43.
Client 13 arrives at 6.63.
Client 14 arrives at 6.94.
Client 15 arrives at 7.83.
Client 16 arrives at 8.05.
Client 5 is finished at 8.27.
Client 6 is processed at 8.27.
Client 6 is finished at 8.84.
Client 7 is processed at 8.84.
Client 17 arrives at 9.22.
Client 18 arrives at 9.54.
Client 19 arrives at 9.56.
Client 20 arrives at 10.14.
Client 7 is finished at 10.74.
Client 8 is processed at 10

In [98]:
data = [0,0,0]

n = 4
lambd = rho * mu * n

runSimulation(n, data)

Server is up
Client 1 arrives at 0.00.
Client 2 arrives at 0.00.
Client 3 arrives at 0.00.
Client 4 arrives at 0.00.
Client 5 arrives at 0.00.
Client 6 arrives at 0.00.
Client 7 arrives at 0.00.
Client 8 arrives at 0.00.
Client 9 arrives at 0.00.
Client 10 arrives at 0.00.
Client 11 arrives at 0.00.
Client 12 arrives at 0.00.
Client 13 arrives at 0.00.
Client 14 arrives at 0.00.
Client 15 arrives at 0.00.
Client 16 arrives at 0.00.
Client 1 is processed at 0.00.
Client 2 is processed at 0.00.
Client 3 is processed at 0.00.
Client 4 is processed at 0.00.
Client 2 is finished at 0.18.
Client 5 is processed at 0.18.
Client 17 arrives at 0.38.
Client 5 is finished at 0.48.
Client 6 is processed at 0.48.
Client 18 arrives at 0.61.
Client 3 is finished at 0.73.
Client 7 is processed at 0.73.
Client 19 arrives at 0.86.
Client 20 arrives at 0.89.
Client 21 arrives at 1.02.
Client 22 arrives at 1.17.
Client 4 is finished at 1.38.
Client 8 is processed at 1.38.
Client 23 arrives at 1.75.
Client 

# Exercise 3

In [99]:
import simpy as sp
import numpy as np
import pandas as pd
import seaborn as sns
import random

RANDOM_SEED = 42
n = 2 # number of servers in the simulation
SERVICETIME = 1.5 # average? minutes to complete a task
T_ARRIVAL = 7 # arrival of a new customer
SIM_TIME = 50 # Simulation length in minutes
rho = 0.5
mu = 2

class Server(object):
    
    def __init__(self, env, num_server):
        
        self.env = env
        self.server = sp.PriorityResource(env, num_server)
        
    def service(self,serviceTime):
        # need to add stochasticity to this
        
        yield self.env.timeout(serviceTime)
        
def randomSample(rate):
    
    return np.random.exponential(rate)
        
def client(env, name, sv, waiting):
    
    arrival = env.now
    print("%s arrives at %.2f." %(name, arrival))
    
    serviceTime = randomSample(SERVICETIME)
    
    with sv.server.request(priority=serviceTime) as request:
        
        yield request
        
        service = env.now
        print("%s is processed at %.2f." % (name, service))
        
        if arrival != service:
            waiting[1] += 1
            waiting[2] += service - arrival
        
        yield env.process(sv.service(serviceTime))
        
        print("%s is finished at %.2f." %(name, env.now))
        
        
def setup(env, num_servers, servicetime, t_arrival, waiting):
    
    server = Server(env, num_servers)
    
    # create 4 customers
    for i in range(4*num_servers):
        waiting[0] += 1
        env.process(client(env, 'Client %d' % waiting[0], server, waiting))
        
    # make more customers while the simulation is running
    while waiting[0]<40*num_servers:
        yield env.timeout(randomSample(1/lambd))
        waiting[0] += 1 
        
        env.process(client(env, 'Client %d' % waiting[0], server, waiting))
        

def runSimulation(serverNumber, waiting):
    
    print('Server is up')
    random.seed(RANDOM_SEED)

    # create the enviroment and start the setup process

    env = sp.Environment()
    env.process(setup(env, serverNumber, SERVICETIME, T_ARRIVAL, waiting))

    env.run()

    print("The total waiting time was %.2f" %(waiting[2]))
    print("There were %i clients, %i of who(m?) had to wait." %(waiting[0],waiting[1]))
    print("The average waiting time was %.2f" %(waiting[2]/waiting[0]))
    
    

In [100]:
waiting = [0,0,0]

n = 1
lambd = rho * mu * n

runSimulation(n, waiting)

Server is up
Client 1 arrives at 0.00.
Client 2 arrives at 0.00.
Client 3 arrives at 0.00.
Client 4 arrives at 0.00.
Client 1 is processed at 0.00.
Client 1 is finished at 1.23.
Client 4 is processed at 1.23.
Client 4 is finished at 1.31.
Client 2 is processed at 1.31.
Client 2 is finished at 2.21.
Client 3 is processed at 2.21.
Client 5 arrives at 2.22.
Client 6 arrives at 3.51.
Client 7 arrives at 4.24.
Client 3 is finished at 4.34.
Client 6 is processed at 4.34.
Client 6 is finished at 4.80.
Client 5 is processed at 4.80.
Client 8 arrives at 4.81.
Client 5 is finished at 5.51.
Client 8 is processed at 5.51.
Client 8 is finished at 6.71.
Client 7 is processed at 6.71.
Client 9 arrives at 6.91.
Client 10 arrives at 8.19.
Client 7 is finished at 9.19.
Client 10 is processed at 9.19.
Client 10 is finished at 9.59.
Client 9 is processed at 9.59.
Client 11 arrives at 11.20.
Client 12 arrives at 12.34.
Client 13 arrives at 12.75.
Client 14 arrives at 13.12.
Client 15 arrives at 13.98.
Clie

In [98]:
waiting = [0,0,0]

n = 2
lambd = rho * mu * n

runSimulation(n, waiting)

Server is up
Client 1 arrives at 0.00.
Client 2 arrives at 0.00.
Client 3 arrives at 0.00.
Client 4 arrives at 0.00.
Client 5 arrives at 0.00.
Client 6 arrives at 0.00.
Client 7 arrives at 0.00.
Client 8 arrives at 0.00.
Client 9 arrives at 0.00.
Client 10 arrives at 0.00.
Client 11 arrives at 0.00.
Client 12 arrives at 0.00.
Client 13 arrives at 0.00.
Client 14 arrives at 0.00.
Client 15 arrives at 0.00.
Client 16 arrives at 0.00.
Client 1 is processed at 0.00.
Client 2 is processed at 0.00.
Client 3 is processed at 0.00.
Client 4 is processed at 0.00.
Client 2 is finished at 0.18.
Client 5 is processed at 0.18.
Client 17 arrives at 0.38.
Client 5 is finished at 0.48.
Client 6 is processed at 0.48.
Client 18 arrives at 0.61.
Client 3 is finished at 0.73.
Client 7 is processed at 0.73.
Client 19 arrives at 0.86.
Client 20 arrives at 0.89.
Client 21 arrives at 1.02.
Client 22 arrives at 1.17.
Client 4 is finished at 1.38.
Client 8 is processed at 1.38.
Client 23 arrives at 1.75.
Client 

In [98]:
waiting = [0,0,0]

n = 4
lambd = rho * mu * n

runSimulation(n, waiting)

Server is up
Client 1 arrives at 0.00.
Client 2 arrives at 0.00.
Client 3 arrives at 0.00.
Client 4 arrives at 0.00.
Client 5 arrives at 0.00.
Client 6 arrives at 0.00.
Client 7 arrives at 0.00.
Client 8 arrives at 0.00.
Client 9 arrives at 0.00.
Client 10 arrives at 0.00.
Client 11 arrives at 0.00.
Client 12 arrives at 0.00.
Client 13 arrives at 0.00.
Client 14 arrives at 0.00.
Client 15 arrives at 0.00.
Client 16 arrives at 0.00.
Client 1 is processed at 0.00.
Client 2 is processed at 0.00.
Client 3 is processed at 0.00.
Client 4 is processed at 0.00.
Client 2 is finished at 0.18.
Client 5 is processed at 0.18.
Client 17 arrives at 0.38.
Client 5 is finished at 0.48.
Client 6 is processed at 0.48.
Client 18 arrives at 0.61.
Client 3 is finished at 0.73.
Client 7 is processed at 0.73.
Client 19 arrives at 0.86.
Client 20 arrives at 0.89.
Client 21 arrives at 1.02.
Client 22 arrives at 1.17.
Client 4 is finished at 1.38.
Client 8 is processed at 1.38.
Client 23 arrives at 1.75.
Client 